In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import jlc
import scipy
import cv2
from utils import (estimate_bias, estimate_bias_coefs,norm_quantile,
                   norm_translation,process_vol,soft_threshold,
                   get_disc_2d,inspect_fiber_vol,filter_vol,
                   get_high_density_mask)
from pathlib import Path
import os

In [2]:
filenames = list(Path("./data/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data\\YCsgA_Exp2_Pos1.tif', 'data\\YCsgA_Exp2_Pos2.tif', 'data\\YCsgA_Exp2_Pos3.tif', 'data\\YCsgA_Exp2_Pos4.tif', 'data\\YCsgA_Exp3_Pos1.tif', 'data\\YCsgA_Exp3_Pos2.tif', 'data\\YCsgA_Exp3_Pos3.tif', 'data\\YCsgA_Exp3_Pos4.tif']


In [3]:
#process and save
if False:
    BEST_TRANS = []
    for filename in filenames:
        savename = filename.replace("data","data_processed2")
        os.makedirs(os.path.dirname(savename),exist_ok=True)
        vol, best_translation = process_vol(filename,smaller=False,reset_origo_on_bad_frame=False)
        jlc.save_tifvol(vol,savename)
        BEST_TRANS.append(best_translation)
        print("done with ",filename)    

In [4]:
#process and save
if True:
    for filename in filenames:
        loadname = filename.replace("data","data_processed3")
        savename = filename.replace("data","data_filtered3")
        os.makedirs(os.path.dirname(savename),exist_ok=True)
        vol = jlc.load_tifvol(loadname)
        vol = filter_vol(vol)
        jlc.save_tifvol(vol,savename)
        print("done with ",filename)    

done with  data\YCsgA_Exp2_Pos1.tif
done with  data\YCsgA_Exp2_Pos2.tif
done with  data\YCsgA_Exp2_Pos3.tif
done with  data\YCsgA_Exp2_Pos4.tif
done with  data\YCsgA_Exp3_Pos1.tif
done with  data\YCsgA_Exp3_Pos2.tif
done with  data\YCsgA_Exp3_Pos3.tif
done with  data\YCsgA_Exp3_Pos4.tif


In [5]:
vol_vis = jlc.load_tifvol(filenames[7].replace("data","data_processed3"))
#vol = filter_vol(vol_vis,strict_max_filter=True)
vol = jlc.load_tifvol(filenames[7].replace("data","data_filtered3"))
jlc.inspect_vol(vol)
jlc.zoom()

In [4]:
# TODO
# Favor newer fibers when figuring out fiber origin
# filter out fibers that moved/dissapeard
# add calculation for fiber direction and include it in fiber origin selection
# dont let fibers do tiny branches
# if a node is very close to an edge and part of the CC, simply adjust the edge

In [61]:
from utils import FiberGraph
fiber_graph = FiberGraph(radius=2,threshold=0.2)
mf = vol.shape[0]
fiber_graph.process_vol(vol,max_frames=mf)
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot()
jlc.zoom()

In [54]:
inspect_fiber_vol(vol_vis,fiber_graph=fiber_graph,vmax=0.5)
jlc.zoom()

In [119]:
len(fiber_graph.edges)

4998

In [121]:
plt.figure()
plt.imshow(vol_vis[mf-1],cmap="gray")
fiber_graph.plot(t=mf+1)
jlc.zoom()

In [65]:
print(fiber_graph.num_fibers())
fiber_graph.cc_crop()
fiber_graph.leaf_crop()
fiber_graph.high_dens_crop(get_high_density_mask(vol[mf-1]))
fiber_graph.remove_inactive()
print(fiber_graph.num_fibers())
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot(t=mf+1)
jlc.zoom()

428
369


In [79]:
paths,paths_cc,path_mean_min_dist,dist = fiber_graph.find_long_seperated_paths(max_dist=10,min_len=10)

found n=49 candidate paths with len>=10
found n=31 candidate paths with mean min dist>10


In [138]:
stats, growth_rate, paths = fiber_graph.estimate_stats(vol_vis[-1]-vol_vis[0].mean(),high_dens_image=get_high_density_mask(vol[-1]),max_dist=5,min_len=10)
stats

found n=49 candidate paths with len>=10
found n=49 candidate paths with mean min dist>5


{'intensity_per_length': 0.7384208990149322,
 'intensity_of_long_paths': 2195.9082441588357,
 'total_intensity': 19541.936869197933,
 'low_intensity': 13475.085725093148,
 'len_of_long_paths': 2973.7894026133604,
 'total_length_intensity': 26464.495919965502,
 'low_length_graph': 24648.411024830122,
 'low_length_intensity': 18248.516182395666,
 'avg_growth_rate': 0,
 'high_dens_ratio': 0.028655653932338633}

In [137]:
vol_vis.mean()

0.08566095429556825

In [140]:
plt.imshow(vol_vis[-1]-vol_vis[0].mean(),vmin=-0.2,vmax=0.2)
plt.colorbar()

In [142]:
for dist,time in growth_rate:
    plt.plot(np.cumsum(time),np.cumsum(dist))

In [112]:
time2 = []
dist2 = []
for dist,time in growth_rate:
    dist2 += dist.tolist()
    time2 += time.tolist()
nonzero_mask = np.array(time2)>0
rate = np.array(dist2)[nonzero_mask]/np.array(time2)[nonzero_mask]
#plt.scatter(time2,dist2)
plt.hist(rate,100)

(array([65., 58., 53., 52., 26., 31., 28., 15., 12., 22., 11.,  9., 12.,
        11.,  8.,  6.,  8., 10.,  6.,  6.,  8.,  6.,  5.,  1.,  6.,  1.,
         4.,  4.,  2.,  7., 13.,  0.,  2.,  0.,  3.,  6.,  0.,  1.,  0.,
         0.,  6.,  2.,  0.,  4.,  2.,  1.,  0.,  0.,  5.,  3.,  0.,  0.,
         2.,  0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,
         4.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         1.,  0.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.]),
 array([ 0.01655643,  0.11837125,  0.22018608,  0.32200091,  0.42381573,
         0.52563056,  0.62744538,  0.72926021,  0.83107504,  0.93288986,
         1.03470469,  1.13651951,  1.23833434,  1.34014917,  1.44196399,
         1.54377882,  1.64559364,  1.74740847,  1.8492233 ,  1.95103812,
         2.05285295,  2.15466777,  2.2564826 ,  2.35829743,  2.46011225,
         2.56192708,  2.6637419 ,  2.76555673,  2.86737156,  2.969186

In [93]:
jlc.inspect_vol(vol_vis-vol_vis[0].mean(),cmap="viridis",vmin=0,vmax=0.4)

In [80]:


plt.imshow(dist,vmin=0,vmax=20)
for path in paths:
    path_xy = fiber_graph.XY[path]
    plt.plot(path_xy[:,0],path_xy[:,1],c="r")
jlc.zoom()

In [52]:
plt.imshow(vol[mf-1],cmap="gray")
for path in paths:
    path_xy = fiber_graph.XY[path]
    plt.plot(path_xy[:,0],path_xy[:,1],c="r")
jlc.zoom()

In [25]:
max([n["cc_idx"] for n in fiber_graph.nodes])

444

In [33]:
#fiber_graph.leaf_crop()
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot(t=mf+1)
jlc.zoom()

In [35]:
fiber_graph.cc_crop(10)
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot(t=mf+1)
jlc.zoom()

In [56]:

#mask = scipy.ndimage.gaussian_filter(vol[mf-1],sigma=5)>0.4
mask = get_high_density_mask(vol[mf-1])
alpha = 0.5
im = vol[mf-1][None].repeat(3,axis=0)
im[0] = mask*alpha+im[0]*(1-mask*alpha)
im[1] = im[1]*(1-mask*alpha)
im[2] = im[2]*(1-mask*alpha)
im = im.transpose(1,2,0)
plt.imshow(im,cmap="gray")
jlc.zoom()

In [10]:
print(max(sum([n["conn"] for n in fiber_graph.nodes],[])))
print(len(fiber_graph.nodes))

3977
3396


In [90]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed")).transpose(0,2,1)
#vol = vol[:,200:300,200:350]
#vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, 1, 1))
#nereast
vol = scipy.ndimage.median_filter(vol, size=(9,1,1), mode='mirror')

In [91]:
def laplace(x,s1=0.75,s2=1.5,clip0=True):
    x = scipy.ndimage.gaussian_filter(x, sigma=(0, s1, s1))-scipy.ndimage.gaussian_filter(x, sigma=(0, s2, s2))
    if clip0:
        x[x<0] = 0
    return x

vol = np.maximum(norm_quantile(laplace(vol),0.01)*soft_threshold(vol,0.08,0.12),soft_threshold(vol,0.5,0.75))
jlc.inspect_vol(vol)
jlc.zoom()

In [ ]:
BEST_TRANS = []
for filename in [filenames[i] for i in [0,2,4,6]]:
    vol, best_translation = process_vol(filename,smaller=True)
    BEST_TRANS.append(best_translation)
    print("done with ",filename)